In [79]:
import numpy as np

In [80]:
#model environment parameters
adjustment_interval_min = 15
charging_duration_min = 120
min_charging_rate_kW = 0
max_charging_rate_kW = 22
battery_capacity_kWh = 32

expected_usage_kWh = 30
sigma_kWh = 5

#optimizer (Reinforcement Learner) parameters
fully_exhousted_penalty = 0#-10_000_000_000 #penalty for charging can get very high itself (-329_703_933)

#computed parameters
num_charging_adjustments = int(charging_duration_min/adjustment_interval_min)


A = np.ones(num_charging_adjustments)

print(num_charging_adjustments)
print(A)

8
[1. 1. 1. 1. 1. 1. 1. 1.]


In [81]:
samples = np.random.normal(expected_usage_kWh, sigma_kWh, 10)
desired_charge = 30
print(samples)

[28.38456659 29.81964437 31.85632139 25.80511756 26.7650622  28.0836349
 31.14146817 23.20511855 31.30540357 32.84705843]


In [82]:
def calcChargingCost(t, p):
    e = np.exp(1)
    return A[t]*(e**p)

def rewardF(totalChargeByInterval, requiredkWh):
    totalCharged = totalChargeByInterval/(int(60/adjustment_interval_min))
    # diviation penalty
    divitation = abs(requiredkWh - totalCharged)
    divitationCost = max(divitation, .001)
    # extra penalty for running out
    runOutCost = fully_exhousted_penalty if requiredkWh > totalCharged else 0
    return  divitationCost + runOutCost

print(rewardF(120, 30))
print(rewardF(110, 30))

0.001
2.5


In [83]:
#generate env

def convertActionToChargingPower(value):
    if value == 0:
        return 0
    if value == 1:
        return 8
    if value == 2:
        return 16
    if value == 3:
        return 22

class env:
    def __init__(self):
        self.nA = 4
        self.nS = self.totalNodes(num_charging_adjustments)
        self.P = self.create_P()
        """ 
        print(self.nS)
        print(self.nA)
        print(self.P) """
        self.current_state = 0

    def reset(self):
        self.current_state = 0
        return np.array([0])

    def step(self, action):
        prob, next_state, reward, done = self.P[self.current_state][action][0]
        self.current_state = next_state
        return (np.array([next_state]), reward, done, prob)

    def totalNodes(self, layer):
        totalNodes = 0
        for lvl in range(0, layer):
            totalNodes = totalNodes + self.nA**lvl
        return totalNodes

    def create_P(self):
        P = []

        numberOfActions = self.nA
        
        def getActionValues(level, levelId):
            actionValue = levelId%numberOfActions
            values = [actionValue]
            ids = [levelId]
            for lvl in range(0, level):
                lastActionId = ids[0]//numberOfActions
                lastActionValue = lastActionId%numberOfActions
                values.insert(0, lastActionValue)
                ids.insert(0, lastActionId)
            return values

        actions = list()

        states = list()

        print("Actions", self.totalNodes(num_charging_adjustments))

        for actionLevel in range(0, num_charging_adjustments + 1):#+1 because the root of the tree has no pretaken action
            lastLayerActions = self.totalNodes(actionLevel)
            layerActions = self.totalNodes(actionLevel+1)

            states.append([])

            for levelActionId in range(0, numberOfActions**actionLevel):
                # levelActionId:
                #0 0 0 0
                #  1 1 1
                #    2 2
                #      3
                #
                # globalIds:
                #0 1 3 6
                #  2 4 7
                #    5 8
                #      9
                #

                #construct location in state tree
                action = levelActionId%numberOfActions
                globalActionId = (lastLayerActions-1) + levelActionId
                actions.append(0)

                originStateLevelId = levelActionId//numberOfActions
                originStateId = originStateLevelId+self.totalNodes(actionLevel-1)
                destinationStateId = self.totalNodes(actionLevel)+(originStateLevelId*numberOfActions)+action

                if(originStateId == len(P)):
                    P.append([])
                elif(originStateId > len(P)):
                    print("!Skipped state!")


                if(actionLevel != num_charging_adjustments):
                    #get total charging power
                    actionValues = getActionValues(actionLevel, levelActionId)
                    #remove root node
                    actionValues = actionValues[1:]
                    charge = 0
                    for i, action in enumerate(actionValues):
                        charge = charge + convertActionToChargingPower(action)
                    rewardValue = 0
                    #limit charge to battery capacity
                    if(charge/4 < battery_capacity_kWh):
                        rewardValue = -(calcChargingCost(actionLevel - 1, convertActionToChargingPower(action))/4)

                    actionTransition = ( 1, destinationStateId, int(rewardValue), False)
                    actions[globalActionId] = actionTransition
                    P[originStateId].append([actionTransition])
                else:
                    #get total charging power
                    actionValues = getActionValues(actionLevel, levelActionId)
                    #remove root node
                    actionValues = actionValues[1:]
                    charge = 0
                    for i, action in enumerate(actionValues):
                        charge = charge + convertActionToChargingPower(action)
                    rewardValue = 0
                    #limit charge to battery capacity
                    if(charge/4 < battery_capacity_kWh):
                        rewardValue = -(calcChargingCost(actionLevel - 1, convertActionToChargingPower(action))/4)
                    rewardValue = rewardF(charge, desired_charge) + rewardValue
                    
                    actionTransition = (1, destinationStateId, int(rewardValue), True)
                    actions[globalActionId] = actionTransition
                    P[originStateId].append([actionTransition])
        #remove root node
        P[0] = P[0][1:]
        return P

env = env()

Actions 21845


## Example Decision Path

In [84]:
# P[n] is the list of states
# each state containes a list of actions P[n][a]
# each action contains a list of tuples like (transition_prob, next_state, reward, done)
# since transition_prob is always 1 there is also only one tuple per action

# take example path

# charge_level_test = [3,3,3,3,3,3,3,3] # This will stop giving negative rewards after some actions when the battery is full
charge_level_test = [2,2,2,2,2,2,2,2] # This will have a very large penalty in the last action when the needed output is not met
# charge_level_test = [2,2,2,3,3,2,2,2] # This will have lower penalty in the end

for prob, next_state, reward, done in env.P[0][charge_level_test[0]]:
    print("1. Action: ", prob, next_state, reward, done)
    for prob, next_state, reward, done in env.P[next_state][charge_level_test[1]]:
        print("2. Action: ", prob, next_state, reward, done)
        for prob, next_state, reward, done in env.P[next_state][charge_level_test[2]]:
                print("3. Action: ", prob, next_state, reward, done)
                for prob, next_state, reward, done in env.P[next_state][charge_level_test[3]]:
                        print("4. Action: ", prob, next_state, reward, done)
                        for prob, next_state, reward, done in env.P[next_state][charge_level_test[4]]:
                                print("5. Action: ", prob, next_state, reward, done)
                                for prob, next_state, reward, done in env.P[next_state][charge_level_test[5]]:
                                        print("6. Action: ", prob, next_state, reward, done)
                                        for prob, next_state, reward, done in env.P[next_state][charge_level_test[6]]:
                                                print("7. Action: ", prob, next_state, reward, done)
                                                for prob, next_state, reward, done in env.P[next_state][charge_level_test[7]]:
                                                    print("8. Action: ", prob, next_state, reward, done)


1. Action:  1 3 -300651 False
2. Action:  1 15 -300651 False
3. Action:  1 63 -300651 False
4. Action:  1 255 -300651 False
5. Action:  1 1023 -300651 False
6. Action:  1 4095 -300651 False
7. Action:  1 16383 -300651 False
8. Action:  1 65535 -300649 True


In [85]:
# build the agent
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.optimizers.legacy import Adam

class ReplayBuffer(object):
    def __init__(self, max_size, input_shape, n_actions, discrete=False):
        self.mem_size = max_size
        self.mem_cntr = 0
        self.discrete = discrete
        self.state_memory = np.zeros((self.mem_size, input_shape))
        self.new_state_memory = np.zeros((self.mem_size, input_shape))
        dtype = np.int8 if self.discrete else np.float32
        self.action_memory = np.zeros((self.mem_size, n_actions), dtype=dtype)
        self.reward_memory = np.zeros(self.mem_size)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.float32)

    def store_transition(self, state, action, reward, state_, done):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        # store one hot encoding of actions, if appropriate
        if self.discrete:
            actions = np.zeros(self.action_memory.shape[1])
            actions[action] = 1.0
            self.action_memory[index] = actions
        else:
            self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.terminal_memory[index] = 1 - done
        self.mem_cntr += 1

    def sample_buffer(self, batch_size):
        max_mem = min(self.mem_cntr, self.mem_size)
        batch = np.random.choice(max_mem, batch_size)

        states = self.state_memory[batch]
        actions = self.action_memory[batch]
        rewards = self.reward_memory[batch]
        states_ = self.new_state_memory[batch]
        terminal = self.terminal_memory[batch]

        return states, actions, rewards, states_, terminal

def build_dqn(lr, n_actions, input_dims, fc1_dims, fc2_dims):
    model = Sequential([
                Dense(fc1_dims, input_shape=(input_dims,)),
                Activation('relu'),
                Dense(fc2_dims),
                Activation('relu'),
                Dense(n_actions)])

    model.compile(optimizer=Adam(lr=lr), loss='mse')

    return model

class Agent(object):
    def __init__(self, alpha, gamma, n_actions, epsilon, batch_size,
                 input_dims, epsilon_dec=0.996,  epsilon_end=0.01,
                 mem_size=1000000, fname='dqn_model.h5'):
        self.action_space = [i for i in range(n_actions)]
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_dec = epsilon_dec
        self.epsilon_min = epsilon_end
        self.batch_size = batch_size
        self.model_file = fname
        self.memory = ReplayBuffer(mem_size, input_dims, n_actions,
                                   discrete=True)
        self.q_eval = build_dqn(alpha, n_actions, input_dims, 256, 256)

    def remember(self, state, action, reward, new_state, done):
        self.memory.store_transition(state, action, reward, new_state, done)

    def choose_action(self, state):
        state = state[np.newaxis, :]
        rand = np.random.random()
        if rand < self.epsilon:
            action = np.random.choice(self.action_space)
        else:
            actions = self.q_eval.predict(state)
            action = np.argmax(actions)

        return action

    def learn(self):
        if self.memory.mem_cntr > self.batch_size:
            state, action, reward, new_state, done = \
                                          self.memory.sample_buffer(self.batch_size)

            action_values = np.array(self.action_space, dtype=np.int8)
            action_indices = np.dot(action, action_values)

            q_eval = self.q_eval.predict(state)

            q_next = self.q_eval.predict(new_state)

            q_target = q_eval.copy()

            batch_index = np.arange(self.batch_size, dtype=np.int32)

            q_target[batch_index, action_indices] = reward + \
                                  self.gamma*np.max(q_next, axis=1)*done

            _ = self.q_eval.fit(state, q_target, verbose=0)

            self.epsilon = self.epsilon*self.epsilon_dec if self.epsilon > \
                           self.epsilon_min else self.epsilon_min

In [86]:
# run learning process
import numpy as np

lr = 0.001
n_games = 200
agent = Agent(gamma=0.99, epsilon=0.0, alpha=lr, input_dims=1,
                n_actions=4, mem_size=1000000, batch_size=64, epsilon_end=0.0)

scores = []
eps_history = []

#env = wrappers.Monitor(env, "tmp/lunar-lander-6",
#                         video_callable=lambda episode_id: True, force=True)

for i in range(n_games):
    done = False
    score = 0
    observation = env.reset()

    while not done:
        action = agent.choose_action(observation)
        observation_, reward, done, info = env.step(action)
        score += reward
        agent.remember(observation, action, reward, observation_, int(done))
        observation = observation_
        agent.learn()
        #ouput the last 5 episodes actions
        if(i > n_games - 6):
            print(action)

    eps_history.append(agent.epsilon)
    scores.append(score)

    avg_score = np.mean(scores[max(0, i-100):(i+1)])
    print('episode: ', i,'score: %.2f' % score,
            ' average score %.2f' % avg_score)


1/1 [==============================] - 0s 12ms/step
episode:  0 score: 29.00  average score 29.00
1/1 [==============================] - 0s 9ms/step
episode:  1 score: 29.00  average score 29.00
1/1 [==============================] - 0s 9ms/step
episode:  2 score: 29.00  average score 29.00
1/1 [==============================] - 0s 8ms/step
episode:  3 score: 29.00  average score 29.00
1/1 [==============================] - 0s 8ms/step
episode:  4 score: 29.00  average score 29.00
1/1 [==============================] - 0s 8ms/step
episode:  5 score: 29.00  average score 29.00
1/1 [==============================] - 0s 8ms/step
episode:  6 score: 29.00  average score 29.00
1/1 [==============================] - 0s 8ms/step
episode:  7 score: 29.00  average score 29.00
2/2 [==============================] - 0s 1ms/step
episode:  8 score: -329703909.00  average score -36633741.89
2/2 [==============================] - 0s 798us/step
episode:  9 score: 29.00  average score -32970364.80
2/2 [